<a href="https://colab.research.google.com/github/DoniaBatool/RAG_2/blob/main/RAG2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

!pip install -qU langchain_community langchain-google-genai langchain-pinecone
!pip install -qU pdfplumber pytube beautifulsoup4 pyyaml pillow pandas requests yt-dlp youtube-transcript-api


In [3]:
!pip install -q pandas==2.2.2  # Fix pandas version

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 90.9 MB/s eta 0:00:00


In [4]:
import os
from google.colab import userdata
from langchain_community.document_loaders import (
    PyPDFLoader,
    TextLoader,
    CSVLoader,
    UnstructuredURLLoader,
    UnstructuredMarkdownLoader,
    UnstructuredImageLoader,
    YoutubeLoader,
    WikipediaLoader,
    GitLoader
)
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_pinecone import PineconeVectorStore
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from pinecone import Pinecone, ServerlessSpec
from uuid import uuid4
from langchain_core.documents import Document
from youtube_transcript_api import YouTubeTranscriptApi

In [5]:
pinecone_api_key = userdata.get('PINECONE_API_KEY')
google_api_key = userdata.get('GOOGLE_API_KEY')

os.environ['GOOGLE_API_KEY'] = google_api_key

In [6]:
pc = Pinecone(api_key=pinecone_api_key)
index_name = "rag-langchain-pinecone-gemini"
pc.create_index(
    name=index_name,
    dimension=768,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)
index = pc.Index(index_name)

In [7]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)
vector_store = PineconeVectorStore(index=index, embedding=embeddings)


In [9]:
# 🎯 YouTube Video ID Extract karein
video_url = "https://www.youtube.com/watch?v=d-VKYF4Zow0&t=2688s"
video_id = video_url.split("v=")[1].split("&")[0]

# 📜 Transcript fetch karein aur `yt_document` mein save karein
try:
    transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['en'])
    yt_document = [text['text'] for text in transcript]  # Transcript ko ek list mein store karna

    print("\n".join(yt_document))
except Exception as e:
    print("Error:", e)


in this course you will learn how to
build and deploy a rag chatbot trained
on your data Ana Kubo will teach you how
to implement the chatbot with Lang chain
JS nextjs versel and open AI you'll also
use data Stacks Who provided a grant to
make this course possible if you're a
beginner look to get your head around AI
concept such as rag Bots vector
embeddings and Technologies such as L
chain this is the perfect course for you
my name is anel Kubo and I'm a software
developer as well as course creator here
on free Camp as well as on my own
channel and today I'm going to be your
guide into the world of generative AI
for JavaScript developers by the end of
this video you won't just have a
fundamental knowledge of the basics of
AI development in JavaScript you also
have your own Formula 1 chatbot that can
answer questions about the famous C
sport that will give you the most
upto-date answers you see chat GPT has a
knowledge cut off date of September 2021
that's when open AI stopped collecti

In [12]:
!git clone https://github.com/Danielskry/Awesome-RAG.git
from langchain.document_loaders import GitLoader

# Local cloned repository ka path
repo_path = "./Awesome-RAG"

git_loader = GitLoader(
    repo_path=repo_path,
    branch="main"
)

git_document= git_loader.load()

# 🔹 Total documents count
print(f"✅ Total Loaded Documents: {len(git_document)}\n")

# 🔹 Har document ka **poora content** print karein
for idx, doc in enumerate(git_document):
    print(f"📄 Document {idx+1}: {doc.metadata['source']}\n")
    print(doc.page_content)  # ✅ Poora data print karega
    print("="*100)  # Separator line


fatal: destination path 'Awesome-RAG' already exists and is not an empty directory.
✅ Total Loaded Documents: 2

📄 Document 1: LICENSE

Creative Commons Legal Code

CC0 1.0 Universal

    CREATIVE COMMONS CORPORATION IS NOT A LAW FIRM AND DOES NOT PROVIDE
    LEGAL SERVICES. DISTRIBUTION OF THIS DOCUMENT DOES NOT CREATE AN
    ATTORNEY-CLIENT RELATIONSHIP. CREATIVE COMMONS PROVIDES THIS
    INFORMATION ON AN "AS-IS" BASIS. CREATIVE COMMONS MAKES NO WARRANTIES
    REGARDING THE USE OF THIS DOCUMENT OR THE INFORMATION OR WORKS
    PROVIDED HEREUNDER, AND DISCLAIMS LIABILITY FOR DAMAGES RESULTING FROM
    THE USE OF THIS DOCUMENT OR THE INFORMATION OR WORKS PROVIDED
    HEREUNDER.

Statement of Purpose

The laws of most jurisdictions throughout the world automatically confer
exclusive Copyright and Related Rights (defined below) upon the creator
and subsequent owner(s) (each and all, an "owner") of an original work of
authorship and/or a database (each, a "Work").

Certain owners wish to 

In [13]:
all_documents = (

    yt_document +

    git_document
)

print(f"Total documents loaded: {len(all_documents)}")

Total documents loaded: 2382


In [19]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

# `git_document` ke objects `Document` hain, magar `yt_document` sirf strings hain
# Isliye humein unhe bhi `Document` format me convert karna hoga
yt_docs = [Document(page_content=text) for text in yt_document]

# Merge YouTube & GitHub documents
all_documents = yt_docs + git_document

# Split documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_documents = text_splitter.split_documents(all_documents)

# ✅ Check output
print(f"📌 Total split chunks: {len(split_documents)}")
print(f"🔹 First chunk:\n{split_documents[0].page_content}")



📌 Total split chunks: 2462
🔹 First chunk:
in this course you will learn how to


In [20]:
uuids = [str(uuid4()) for _ in range(len(split_documents))]
vector_store.add_documents(documents=split_documents, ids=uuids)

['ad719cd0-ff61-4381-85ca-6b72aed91d1d',
 '1ce40575-0432-48a8-b6e2-4c41cd208a95',
 'fef21e04-c670-4e62-bcee-0c09243b3c71',
 'a178326f-5015-4c8b-b143-ce4a80981a54',
 'e3086187-a29c-49c9-9442-28759bb85377',
 '78bb1ab3-9539-408d-9520-f147791fce74',
 '49ed1b4f-4c7e-488e-82bd-b22d95673922',
 'e45ff612-cc9a-4f41-8a21-46427503b046',
 'd77e38f1-70ff-459e-b36a-61b7fad87ed5',
 'e248d865-a9b7-4940-8d51-4faf762bc914',
 '64f216cd-f208-49bd-ac3d-17cc54852728',
 '542659ff-b94f-4ff4-a83d-383e2d1d5da7',
 'adbf4f91-3c9e-4f7b-b586-eb274588b278',
 '80da79d4-0dab-471a-b42f-8a4bed803bab',
 '706ed4fb-1639-42fe-ae25-e05974cd9c52',
 'a48e8244-7f84-4403-9545-88a46fbba95b',
 '3a2803f5-da61-42ed-82cf-a309d39dbdec',
 'ab1f99f3-0ef9-4946-a572-c95781a936d5',
 '2cad94f8-9726-4fd1-a8e3-ba0dd47344e2',
 'bc542de7-456b-4e50-bbe9-b1ab3566098d',
 '7bf5ff15-10bc-4a39-b652-e90b6b88d1eb',
 '28f55d8c-a309-41f3-963f-dfef6fda01e7',
 '82f75b6c-7a9d-4da8-aca4-7b0267e292d7',
 '11ff014d-6be2-4926-9f31-b2276ddac74e',
 'dc9a8d90-8461-

In [21]:
def answer_to_question(query: str):
    vector_results = vector_store.similarity_search(query, k=2)
    final_answer = llm.invoke(f"Answer this user query: {query}, based on the context: {vector_results}")
    return final_answer


In [22]:
answer = answer_to_question("What is a RAG system?")
print(answer.content)


Based on the provided text, a RAG system is a system whose implementations vary in complexity.  They range from simple document retrieval to more advanced methods using iterative feedback loops and specialized improvements.  A typical RAG implementation involves a series of key steps (though these steps are not detailed in the provided text).
